Imports

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
%matplotlib inline

Read CSV and save as a data frame

In [2]:
df = pd.read_csv('911.csv')

Useful calls for data frames

In [ ]:
df.info()

df.head()
#Useful for quick checks on it modifications to your df worked. Arg = #rows shown

df['zip'].value_counts() 
#for top five + .head()

df['title'].nunique()

df.idxmin()
#returns the indeces where column values were the minimum 
#min can be changed to max
#df['col1'].argmin() for each column also works

df.loc['startindex': 'endindex'].std()
#returning the standard dev over a slice of the rows


Creating new features

In [6]:
nameme=pd.DataFrame()
#create new, empty, DF

df['Reason'] = df['title'].apply(lambda x: x.split(':')[0])
#creating new reason column based on the first part of the 'title' column value (e.g., EMS: BACK PAINS/INJURY )

df['Date'] = df['timeStamp'].apply(lambda x: x.date())
#Create a new column called that contains the date from the timeStamp column. 

df2['Returns'] = df1['forloopnums']['col1'].pct_change()
#Creating a new column in a second df for the %change in 'col1'. 'col1' is in a multi-level index with 'forloopnums' number of instances

In [ ]:
byMonth = df.groupby('Month').count()
#creating a new groupby object and calling .count() to aggregate


dayHour = df.groupby(by=['Day of Week','Hour']).count()['Reason'].unstack()
#Restructure  dataframe so that the columns become the Hours (currently 1 column) and the Index becomes the Day of the Week (currently one column)
#unstack takes a multi-level index DF and turns it into matrix form by turning 1 index into column and 1 into index
#by[-1] becomes the new column values with unstack method
#May need df.set_index('Day of Week', inplace= True) to change index

Column object types

In [ ]:

df.info()
#Check types on all columns
type(df['timeStamp'][0])
#Check type on a specific items in a column

df['timeStamp'] = pd.to_datetime(df['timeStamp'])
#reset column to convert the values from strings to DateTime objects

df['Day of Week'] = df['Day of Week'].map({0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'})
#Convert a columnn of #s to strings with a dictionary

x.date()
#Timestamp contains the date and time so if you just want the date convert it to a date 

Grabbing attributes off a DateTime object

In [ ]:
df['Hour'] = df['timeStamp'].apply(lambda time: time.hour)
#Creating new hour column using an attribute of the value in the timestamp column

<b>Data Viz</b>

In [ ]:
#optional imports
import plotly
import cufflinks as cf
cf.go_offline()

In [ ]:
plt.tight_layout()
#make plot prettier

plt.figure(figsize=(12,6))
#can add in cell with sns code

byMonth['twp'].plot()
#creating a simple plot of a dataframe column (the DF is a groupby obj here) 

sns.countplot(x = 'Reason', data=df)
#use seaborn to create a countplot of 911 calls by Reason

sns.countplot(x = 'Day of Week', data = df, hue = 'Reason', palette='viridis')
#use seaborn to create a countplot of 1 column with the hue based off of a second column

sns.histplot(msdata, kde=True, bins=50)
#a distribution plot. msdata=df['col1'].loc[start:end]

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#place legend outside the chart

sns.lmplot( x ='Month', y= 'twp', data = byMonth.reset_index()) 
#seaborn linear model plot that is also resetting the index to make the month a column/not index. 
#reset_index() only necessary if the x or y value is a current index

df.groupby('Date').count()['twp'].plot(lw = 2)
#groupby this Date column with the count() aggregate and create a plot of counts of 911 calls.
#Some difference between column values because not every call has all the information put in, so you choose one column value(e.g., ['twp'])

df[df['Reason']=='Traffic'].groupby('Date').count()['twp'].plot(Title = 'Traffic')
#Use conditional selection to plot only the rows where reason==traffic

closepcorr= closep.corr()
#building correlation DF
#closep is a DF (can be sliced parts of a DF) with columns we want to correlate against one another (for a heatmap)
#pass closepcorr into the arg of the heatmap

sns.heatmap(dayHour, cmap='viridis')
#dayHour must be a dataframe in matrix form (e.g., pivot table)
#In matrix form, every cell is a value at the x,y position

sns.clustermap(dayHour, cmap='coolwarm')
#dayHour is in matrix form

sns.pairplot(returns[1:])
#create a pairplot of a df starting at index=1

df.xs('col1', axis = 1, level='level1').iplot()
# Use plotly and cufflinks to create a line plot for a cross-section of a multi-level DF

Grabbing stock data from online

https://pandas-datareader.readthedocs.io/en/latest/remote_data.html

In [ ]:
from pandas_datareader import data, wb
import datetime

start = pd.to_datetime('01-01-2006')
#Use datetime to set start and end datetime objects

BAC=data.DataReader('BAC', 'yahoo', start, end)
#Use datareader to grab info on the stock.

tickers = 'BAC C GS JPM MS WFC'
tickers = tickers.split()
#Create a list of the column categories you want on the top level (as strings). 

bank_stocks = pd.concat( [BAC, C, GS, JPM, MS, WFC], axis=1, keys=tickers)
#keys= list of column categories
#concatenate the datareader dataframes together (list passed in NOT as strings) into a single data frame
#concat along columns (axis=1) because all DFs share the same index (the date)

bank_stocks.columns.names = ['Bank Ticker','Stock Info']
#Set the column name levels. First index level is [0], second is [1], etc.

Data analysis of stocks

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.xs.html

In [ ]:
bank_stocks.xs('Close', axis = 1, level='Stock Info').max()
#Grab information from within a multilevel index using cross-section (xs)
#Returning the max Close price for each bank's stock 

closep= bank_stocks.xs('Close', axis = 1, level='Stock Info').iplot()
# Create a line plot showing Close price for each bank for the entire index of time. 



In [ ]:
f = plt.figure(figsize=(12,6))
bank_stocks.xs('Close', axis = 1, level='Stock Info')['BAC'].loc['01-01-2008': '01-01-2009'].rolling(30).mean().plot(label = 'Rolling AVG')
bank_stocks.xs('Close', axis = 1, level='Stock Info')['BAC'].loc['01-01-2008': '01-01-2009'].plot(label = 'BAC Close')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
# Plot the rolling 30 day average against the Close Price for Bank Of America's stock for the year 2008

In [ ]:
closep = bank_stocks.xs('Close', axis = 1, level='Stock Info')
closepcorr= closep.corr()
sns.heatmap(closepcorr, cmap='coolwarm')
#Create a heatmap of the correlation between the stocks Close Price

Cufflinks for Technical Data Analysis

In [ ]:
BAC[['High', 'Low', 'Open', 'Close']].loc['01-01-2015': '01-01-2016'].iplot(kind='candle')
#Use .iplot(kind='candle) to create a candle plot of Bank of America's stock from Jan 1st 2015 to Jan 1st 2016.
#BAC is a DF reading financial info from yahoo

df.ta_plot(study='sma') 
#create a Simple Moving Averages plot 

df..ta_plot(study='boll') 
#create a Bollinger Band Plot 